In [ ]:
# 목표

# 승객이 타이타닉 침몰에서 살아남았는지 아닌지를 예측하는 것은 당신의 일입니다.
# 테스트 세트의 각에 대해 변수에 대해 0 또는 1 값을 예측해야 합니다.

# 미터법
# 당신의 점수는 당신이 정확하게 예측하는 승객의 비율입니다. 이것은 부정확성으로 알려져 있다.

# 제출 파일 형식
# 정확히 418개의 항목과 헤더 행이 있는 csv 파일을 제출해야 합니다. 추가 열(PassingsingId 및 Survived 이상) 또는 행이 있는 경우 제출 시 오류가 표시됩니다.

# 파일은 정확히 2개의 열이 있어야 합니다.

# PassengerId (모든 순서로 정렬)
# 생존 (당신의 이진 예측 포함: 생존은 1, 사망한 것은 0)
# 승객 Id, 생존
# 892,0
# 893,1
# 894,0
# 등등.


In [ ]:
# pseudo code(의사코드) (무엇을 한다 > 어떤 함수나 로직을 사용을 할 것인지)
# 1. 라이브러리 불러오기


# 2. 데이터 불러오기

# 3. 데이터 전처리

# 4. 특징 공학

# 5. 모델 학습

# 6. 평가

# 7. 제출


In [1]:
import os
import pandas as pd

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

# 0) Kaggle 환경인지 로컬 환경인지 구분
if os.path.exists('/kaggle/input/titanic'):
    DATA_DIR = '/kaggle/input/titanic'
else:
    DATA_DIR = os.path.expanduser(
        '/Users/rick/Library/Mobile Documents/com~apple~CloudDocs/'
        'Study/통계&빅데이터/Kaggle/titanic/data/'
    )

# 1) 원본 데이터 로드
train    = pd.read_csv(os.path.join(DATA_DIR, 'train.csv'))
raw_test = pd.read_csv(os.path.join(DATA_DIR, 'test.csv'))

# 2) 제출용 PassengerId 저장
passenger_ids = raw_test['PassengerId']

# 3) test_features 생성 (drop 시 컬럼이 없으면 무시)
test_features = raw_test.drop(
    columns=['PassengerId', 'Name', 'Ticket', 'Cabin'],
    errors='ignore'
)

# 4) train에서도 불필요 컬럼(PassengerId 포함) 제거
train = train.drop(
    columns=['PassengerId', 'Name', 'Ticket', 'Cabin'],
    errors='ignore'
)

# 5) 결측치 처리용 파이프라인 준비
numeric_features     = ['Age', 'SibSp', 'Parch', 'Fare']
categorical_features = ['Pclass', 'Sex', 'Embarked']

numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median'))
])

categorical_transformer = OneHotEncoder(
    drop='first',
    sparse_output=False
)

preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
], remainder='passthrough')

# 6) 전체 파이프라인: 전처리 → 로지스틱 회귀
pipeline = Pipeline([
    ('preproc', preprocessor),
    ('clf', LogisticRegression(
        solver='liblinear',
        penalty='l2',
        C=1.0,
        max_iter=1000,
        random_state=42
    ))
])

# 7) 교차검증으로 대략적 성능 확인
X = train.drop(columns='Survived', errors='ignore')
y = train['Survived']
cv_scores = cross_val_score(pipeline, X, y, cv=5, scoring='accuracy')
print(f"5-폴드 CV 정확도: {cv_scores.mean():.3f} ± {cv_scores.std():.3f}")

# 8) 전체 train 데이터로 학습
pipeline.fit(X, y)

# 9) test 데이터 예측
test_preds = pipeline.predict(test_features)

# 10) submission.csv 생성
submission = pd.DataFrame({
    'PassengerId': passenger_ids,
    'Survived':      test_preds
})
submission.to_csv('submission.csv', index=False)
print("✔ submission.csv 파일이 생성되었습니다.")

5-폴드 CV 정확도: 0.797 ± 0.023
✔ submission.csv 파일이 생성되었습니다.


In [ ]:
# 11) submission.csv 확인
# 결과값 확인
import pandas as pd

# 1) 파일 읽기
sub = pd.read_csv('submission.csv')

# 2) 상위 5개 행 출력
print("---- submission.csv 상위 5개 ----")
display(sub.head())

# 3) 행 개수 확인 (test 데이터와 일치해야 함)
print(f"submission 행 수: {len(sub)}")
print(f"test_features 행 수: {len(test_features)}")  # 이미 로드된 test_features 사용

# 4) 결측치 확인
print("\n결측치 확인:")
print(sub.isnull().sum())

# 5) 컬럼명 확인
print("\n컬럼명:")
print(sub.columns.tolist())